In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [5]:
df_train=pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv',index_col=0)
df_train.head()


,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [6]:
df_train_num=df_train[['days_left','duration']]
df_train_num.head()

,days_left,duration
id,,
1,21,14.25
2,7,1.75
3,5,9.58
4,28,6.75
5,4,2.00


In [8]:
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
num_scaler=std_scaler.fit_transform(df_train_num)
num_scaler

array([[-0.36141789,  0.28952803],
       [-1.38897606, -1.45682723],
       [-1.53577008, -0.36291029],
       ...,
       [-0.65500594,  3.23458153],
       [-0.36141789,  0.58012154],
       [ 0.00556716,  0.84836171]])

In [9]:
df_train_cat=df_train[['stops','destination_city','class']]
df_train_cat.head()

,stops,destination_city,class
id,,,
1,one,Mumbai,Economy
2,zero,Kolkata,Economy
3,one,Delhi,Business
4,one,Hyderabad,Economy
5,zero,Mumbai,Economy


In [10]:
y=df_train['price']
y.head()

id
1     7212
2     5292
3    60553
4     5760
5    10712
Name: price, dtype: int64

In [11]:
df=df_train[['stops','destination_city','class','duration','days_left']]
df.head()

,stops,destination_city,class,duration,days_left
id,,,,,
1,one,Mumbai,Economy,14.25,21
2,zero,Kolkata,Economy,1.75,7
3,one,Delhi,Business,9.58,5
4,one,Hyderabad,Economy,6.75,28
5,zero,Mumbai,Economy,2.00,4


In [12]:
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
num_pipeline = Pipeline([ 
          ('imputer', SimpleImputer(strategy='median')),
          ('std_scaler', StandardScaler())              
])
num_pipeline.fit_transform(df_train_num)

array([[-0.36141789,  0.28952803],
       [-1.38897606, -1.45682723],
       [-1.53577008, -0.36291029],
       ...,
       [-0.65500594,  3.23458153],
       [-0.36141789,  0.58012154],
       [ 0.00556716,  0.84836171]])

In [14]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OrdinalEncoder

num_attribs = list(df_train_num) 
cat_attribs = ['stops']
cat1_attribs=['destination_city']
cat2_attribs=['class']


full_pipeline = ColumnTransformer([ 
    ('num', num_pipeline, num_attribs), 
    ('cat', OneHotEncoder(), cat_attribs),
    ('cat1',OneHotEncoder(),cat1_attribs),
    ('cat2',OneHotEncoder(),cat2_attribs)
])



In [15]:
df_prepared=full_pipeline.fit_transform(df)
df_prepared

array([[-0.36141789,  0.28952803,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.38897606, -1.45682723,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-1.53577008, -0.36291029,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.65500594,  3.23458153,  0.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.36141789,  0.58012154,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.00556716,  0.84836171,  1.        , ...,  0.        ,
         0.        ,  1.        ]])

In [16]:
from sklearn.linear_model import LinearRegression 
LR_model = LinearRegression()

In [17]:
LR_model.fit(df_prepared,y)

LinearRegression()

In [18]:
test_data=df.sample(5)
test_data.head()

,stops,destination_city,class,duration,days_left
id,,,,,
5660,one,Hyderabad,Business,16.25,45
8035,one,Bangalore,Business,9.75,29
6955,one,Chennai,Economy,6.50,9
13248,one,Kolkata,Business,16.17,18
14461,one,Kolkata,Economy,9.92,1


In [19]:
test_label = y.loc[test_data.index]
test_label

id
5660     52175
8035     49177
6955      6369
13248    55377
14461    22608
Name: price, dtype: int64

In [20]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[ 1.40011038,  0.56894487,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ],
       [ 0.2257582 , -0.33915986,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ],
       [-1.24218203, -0.79321223,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ],
       [-0.58160893,  0.55776819,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ],
       [-1.82935813, -0.31540943,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ]])

In [21]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([49726.22525027, 53094.06300974,  9705.3013247 , 55957.18746069,
       12461.47141272])

In [22]:
pd.DataFrame({'Bashorat':predicted_data, 'Real baxosi': test_label})

,Bashorat,Real baxosi
id,,
5660,49726.225250,52175
8035,53094.063010,49177
6955,9705.301325,6369
13248,55957.187461,55377
14461,12461.471413,22608


In [23]:
test_set=pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv',index_col=0)
test_set.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
id,,,,,,,,,,
1,Air_India,AI-765,Kolkata,Evening,one,Night,Delhi,Business,28.25,2
2,Vistara,UK-747,Delhi,Early_Morning,one,Night,Mumbai,Business,13.83,34
3,Air_India,AI-570,Mumbai,Early_Morning,zero,Early_Morning,Chennai,Business,2.00,30
4,AirAsia,I5-974,Hyderabad,Night,one,Late_Night,Delhi,Economy,5.17,26
5,Air_India,AI-770,Kolkata,Night,one,Afternoon,Mumbai,Economy,16.33,35


In [28]:
df_test=test_set[['stops','destination_city','class','duration','days_left']]
df_test

,stops,destination_city,class,duration,days_left
id,,,,,
1,one,Delhi,Business,28.25,2
2,one,Mumbai,Business,13.83,34
3,zero,Chennai,Business,2.00,30
4,one,Delhi,Economy,5.17,26
5,one,Mumbai,Economy,16.33,35
...,...,...,...,...,...
4996,one,Bangalore,Business,17.42,15
4997,zero,Mumbai,Economy,3.00,40
4998,one,Mumbai,Business,11.17,37


In [30]:
test_num=test_set[['duration','days_left']]
test_num.head()

,duration,days_left
id,,
1,28.25,2
2,13.83,34
3,2.00,30
4,5.17,26
5,16.33,35


In [31]:
test_prepared=full_pipeline.fit_transform(df_test)
test_prepared

array([[-1.75411081,  2.17930407,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.58318436,  0.20548051,  1.        , ...,  1.        ,
         1.        ,  0.        ],
       [ 0.29102246, -1.41382134,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.80230578, -0.15862287,  1.        , ...,  1.        ,
         1.        ,  0.        ],
       [ 0.87534626,  0.13704002,  1.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.67879147,  0.51346269,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [33]:
predicted_test = LR_model.predict(test_prepared)
predicted_test

array([56147.91253067, 52191.52263016, 44130.46385434, ...,
       51687.98345371, 50605.6019851 , 49270.46800134])